In [1]:
import json
import os
from datetime import datetime
from repo_code_file import generating_patient_ids, creating_patient_info, saving_data, gettingpatientdata, blood_pressure_adjustments, appointmentdate_data
from unittest.mock import patch

In [2]:
# Path for testing (temporary file)
test_file_path = 'test_patient_info.json'

# Clear any existing test data
if os.path.exists(test_file_path):
    os.remove(test_file_path)

# Recreate the file for testing purposes
with open(test_file_path, 'w') as f:
    json.dump({}, f)  # Start with an empty JSON object

In [3]:
# Test generating_patient_ids function
def test_generating_patient_ids():
    test_data = {1: {"Name": "John Doe"}, 2: {"Name": "Jane Doe"}}
    result = generating_patient_ids(test_data)
    assert result == 3, "Test Failed: Expected ID should be 3"
    print("Test Passed: generating_patient_ids() returns expected ID.")
test_generating_patient_ids()

Test Passed: generating_patient_ids() returns expected ID.


In [4]:
# Define a valid test file path
test_file_path = "test_patient_info.json"


# Test creating_patient_info function only
def test_creating_patient_info():
    # Mock input for testing
    inputs = ['John Doe', 'Male', '01-01-1990', 'Nairobi']
    
    # Redirect input for `creating_patient_info`
    with patch('builtins.input', side_effect=inputs):
        loaded_data = creating_patient_info(file_path=test_file_path)

    # Verify if the generated dictionary has the expected keys and values
    assert "Name(First and Last Name)" in loaded_data[list(loaded_data.keys())[0]], "Test Failed: 'Name' key is missing."
    assert loaded_data[list(loaded_data.keys())[0]]["Name(First and Last Name)"] == "john doe", "Test Failed: Name not saved correctly."
    print("Test Passed: creating_patient_info() works as expected.")



# Run tests
test_creating_patient_info()

Please provide the required information
Test Passed: creating_patient_info() works as expected.


In [5]:
# Test saving_data function
def test_saving_data():
    # Clear the test file if it exists
    if os.path.exists(test_file_path):
        os.remove(test_file_path)
    
    # Mock input for creating patient data
    inputs = ['John Doe', 'Male', '01-01-1990', 'Nairobi']
    
    # Redirect input for creating_patient_info
    with patch('builtins.input', side_effect=inputs):
        # Call saving_data to save data to the file
        saving_data()
    
    # Check if data was saved correctly
    with open(test_file_path) as f:
        saved_data = json.load(f)
    
    # Get the ID key dynamically
    first_key = list(saved_data.keys())[0]
    
    # Verify the data content
    assert len(saved_data) == 1, "Test Failed: Data was not saved correctly."
    assert saved_data[first_key]["Name(First and Last Name)"] == "john doe", "Test Failed: Name not saved correctly."
    print("Test Passed: saving_data() works as expected.")
    
test_saving_data()

Please provide the required information
Data has been saved successfully!
Test Passed: saving_data() works as expected.


In [6]:
file_path = test_file_path

In [7]:
# Add other columns
def gettingpatientdata(file_path=file_path):
    with open(file_path) as f:
        data = json.load(f)
    
    key_searched = input("Patient ID: ")
    patient_found = False

    for key, values in data.items():
        if key == key_searched:
            print(values)  # Open the dashboard with the patient's information
            patient_found = True
            break  # Exit the loop if the patient is found
    
    if not patient_found:
        print("Patient has no such data. Collecting data...")
        saving_data()

In [8]:
# Test appointmentdate_data function
def test_appointmentdate_data():
    inputs = ['1', '20-10-2024']  # Patient ID to search, and new appointment date
    with patch('builtins.input', side_effect=inputs):
        appointmentdate_data(file_path=test_file_path)

    with open(test_file_path) as f:
        saved_data = json.load(f)

    assert saved_data['1']['Next Appointment'] == "20-10-2024", "Test Failed: Next Appointment was not updated."
    print("Test Passed: appointmentdate_data() works as expected.")

test_appointmentdate_data()

Updated appointment details for patient id - 1.
Test Passed: appointmentdate_data() works as expected.


In [9]:
# Test gettingpatientdata function
def test_gettingpatientdata():
    inputs = ['1']  # Patient ID to search
    with patch('builtins.input', side_effect=inputs):
        gettingpatientdata(file_path=test_file_path)

    with open(test_file_path) as f:
        saved_data = json.load(f)
    
    assert "Previous Appointment" in saved_data['1'], "Test Failed: Previous Appointment was not added."
    print("Test Passed: gettingpatientdata() works as expected.")

test_gettingpatientdata()

{'Name(First and Last Name)': 'john doe', 'Gender': 'male', 'YearofBirth(DD-MM-YYYY)': '01-01-1990', 'Age': 34, 'Location': 'nairobi', 'Previous Appointment': None, 'Next Appointment': '20-10-2024'}
Test Passed: gettingpatientdata() works as expected.


In [10]:
from unittest.mock import patch

def test_blood_pressure_adjustments():
    inputs_per_visit = [
        ['1', '120'],  # First visit
        ['1', '130'],  # Second visit
        ['1', '110'],  # Third visit
        ['1', '125'],  # Fourth visit
        ['1', '140'],  # Fifth visit
        ['1', '135']   # Sixth visit (should trigger shifting)
    ]

    for visit_num, inputs in enumerate(inputs_per_visit, start=1):
        with patch('builtins.input', side_effect=inputs):
            blood_pressure_adjustments(file_path=test_file_path)

        with open(test_file_path) as f:
            saved_data = json.load(f)
        
        print(f"Saved data after visit {visit_num}: {saved_data['1']}")  # Debug output
        
        # Check if blood pressure entries are as expected
        if visit_num <= 5:
            # Validate that the correct Blood Pressure entry exists
            assert f'Blood Pressure {visit_num}' in saved_data['1'], \
                f"Test Failed: 'Blood Pressure {visit_num}' should exist after visit {visit_num}."
            assert saved_data['1'][f'Blood Pressure {visit_num}'] == int(inputs[1]), \
                f"Test Failed: Value for 'Blood Pressure {visit_num}' not saved correctly."

        if visit_num == 6:
            # Validate that values have shifted correctly
            expected_values = {
                'Blood Pressure 1': 130,
                'Blood Pressure 2': 110,
                'Blood Pressure 3': 125,
                'Blood Pressure 4': 140,
                'Blood Pressure 5': 135
            }
            for key, expected_value in expected_values.items():
                assert saved_data['1'][key] == expected_value, \
                    f"Test Failed: {key} should be {expected_value} after shifting on sixth visit."
            print("Test Passed: blood_pressure_adjustments() works as expected for 6th visit with shifts.")
    
    print("Test Passed: blood_pressure_adjustments() handles incremental visits correctly.")

test_blood_pressure_adjustments()


Added new entry: Blood Pressure 1 with value 120
Updated Blood Pressure details for patient ID 1.
Saved data after visit 1: {'Name(First and Last Name)': 'john doe', 'Gender': 'male', 'YearofBirth(DD-MM-YYYY)': '01-01-1990', 'Age': 34, 'Location': 'nairobi', 'Previous Appointment': None, 'Next Appointment': '20-10-2024', 'Blood Pressure 1': 120}
Added new entry: Blood Pressure 2 with value 130
Updated Blood Pressure details for patient ID 1.
Saved data after visit 2: {'Name(First and Last Name)': 'john doe', 'Gender': 'male', 'YearofBirth(DD-MM-YYYY)': '01-01-1990', 'Age': 34, 'Location': 'nairobi', 'Previous Appointment': None, 'Next Appointment': '20-10-2024', 'Blood Pressure 1': 120, 'Blood Pressure 2': 130}
Added new entry: Blood Pressure 3 with value 110
Updated Blood Pressure details for patient ID 1.
Saved data after visit 3: {'Name(First and Last Name)': 'john doe', 'Gender': 'male', 'YearofBirth(DD-MM-YYYY)': '01-01-1990', 'Age': 34, 'Location': 'nairobi', 'Previous Appointmen

In [11]:
# Cleanup test data
if os.path.exists(test_file_path):
    os.remove(test_file_path)